In [1]:
import numpy as np

# Define computational basis states
ket_0 = np.array([[1], [0]])  # |0⟩
ket_1 = np.array([[0], [1]])  # |1⟩

# Define POVM measurement elements (projectors for |0⟩ and |1⟩)
E_0 = ket_0 @ ket_0.T  # |0⟩⟨0|
E_1 = ket_1 @ ket_1.T  # |1⟩⟨1|

# Prior probabilities of the system being in |0⟩ or |1⟩ before measurement
prior_0 = 0.6  # Assume 60% chance the system was in |0⟩
prior_1 = 0.4  # Assume 40% chance the system was in |1⟩

# Define possible pre-measurement density matrices
rho_0 = E_0  # |0⟩⟨0|
rho_1 = E_1  # |1⟩⟨1|

# Compute predictive probabilities P(measurement outcome | initial state)
p_0_given_rho_0 = np.trace(E_0 @ rho_0)  # Probability of measuring 0 given state |0⟩
p_0_given_rho_1 = np.trace(E_0 @ rho_1)  # Probability of measuring 0 given state |1⟩

p_1_given_rho_0 = np.trace(E_1 @ rho_0)  # Probability of measuring 1 given state |0⟩
p_1_given_rho_1 = np.trace(E_1 @ rho_1)  # Probability of measuring 1 given state |1⟩

# Compute marginal probabilities P(measurement outcome)
p_0 = p_0_given_rho_0 * prior_0 + p_0_given_rho_1 * prior_1
p_1 = p_1_given_rho_0 * prior_0 + p_1_given_rho_1 * prior_1

# Compute retrodictive probabilities P(initial state | measurement outcome) using Bayes' Rule
p_rho_0_given_0 = (p_0_given_rho_0 * prior_0) / p_0  # P(state |0⟩ was initial state | measured 0)
p_rho_1_given_0 = (p_0_given_rho_1 * prior_1) / p_0  # P(state |1⟩ was initial state | measured 0)

p_rho_0_given_1 = (p_1_given_rho_0 * prior_0) / p_1  # P(state |0⟩ was initial state | measured 1)
p_rho_1_given_1 = (p_1_given_rho_1 * prior_1) / p_1  # P(state |1⟩ was initial state | measured 1)

# Display results
print("Predictive Probabilities (P(outcome | state)):")
print(f"P(measure 0 | state |0⟩) = {p_0_given_rho_0:.2f}")
print(f"P(measure 0 | state |1⟩) = {p_0_given_rho_1:.2f}")
print(f"P(measure 1 | state |0⟩) = {p_1_given_rho_0:.2f}")
print(f"P(measure 1 | state |1⟩) = {p_1_given_rho_1:.2f}")

print("\nRetrodictive Probabilities (P(state | outcome)):")
print(f"P(state |0⟩ was initial | measured 0) = {p_rho_0_given_0:.2f}")
print(f"P(state |1⟩ was initial | measured 0) = {p_rho_1_given_0:.2f}")
print(f"P(state |0⟩ was initial | measured 1) = {p_rho_0_given_1:.2f}")
print(f"P(state |1⟩ was initial | measured 1) = {p_rho_1_given_1:.2f}")


Predictive Probabilities (P(outcome | state)):
P(measure 0 | state |0⟩) = 1.00
P(measure 0 | state |1⟩) = 0.00
P(measure 1 | state |0⟩) = 0.00
P(measure 1 | state |1⟩) = 1.00

Retrodictive Probabilities (P(state | outcome)):
P(state |0⟩ was initial | measured 0) = 1.00
P(state |1⟩ was initial | measured 0) = 0.00
P(state |0⟩ was initial | measured 1) = 0.00
P(state |1⟩ was initial | measured 1) = 1.00


In [2]:
import numpy as np
from scipy.linalg import expm

# Number of qubits
n_qubits = 4
i = 9  # encoding for psi_0
j = 2  # bitflip index for psi_f_noisy
dim = 2 ** n_qubits  # Hilbert space dimension
noise = 0.2

# Define identity and Pauli matrices
I = np.eye(2, dtype=np.complex64)
X = np.array([[0, 1], [1, 0]], dtype=np.complex64)
Z = np.array([[1, 0], [0, -1]], dtype=np.complex64)

# Build multi-qubit Hamiltonians
H0 = sum(np.kron(np.kron(np.eye(2**i), Z), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))
Hf = sum(np.kron(np.kron(np.eye(2**i), X), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))

# Function to evolve quantum state
def time_evolution(H, dt):
    return expm(-1j * H * dt)

def hamiltonian_evolution(psi, H_init, H_final, backwards=False, T=1.0, steps=50):
    dt = T / steps
    psi_t = psi.copy()
    for step in range(steps):
        s = step / steps
        H = (1 - s) * H_init + s * H_final
        U = time_evolution(H, dt if not backwards else -dt)
        psi_t = U @ psi_t
    return psi_t / np.linalg.norm(psi_t)  # Normalize

# Start with initial state |ψ₀⟩
true_psi_0 = np.zeros(dim, dtype=np.complex64)
true_psi_0[i] = 1

# Evolve forward to get |ψ_f⟩
psi_f = hamiltonian_evolution(true_psi_0, H0, Hf)

# Define multiple noisy versions of Hf
noise_models = [
    Hf + noise * np.random.randn(*Hf.shape),  # Gaussian noise
    Hf + noise * np.random.uniform(-1, 1, Hf.shape),  # Uniform noise
    Hf + 0.1 * np.eye(dim)  # Small identity perturbation
]

# Store fidelity scores
fidelity_scores = []

# Test each noisy model
for Hn in noise_models:
    psi_f_noisy = hamiltonian_evolution(true_psi_0, H0, Hn)
    fidelity = np.abs(np.vdot(psi_f_noisy, psi_f))**2
    fidelity_scores.append(fidelity)

# Bayesian weighting of the best noise model
likelihoods = np.exp(np.array(fidelity_scores) / sum(fidelity_scores))  # Normalize
best_Hn = noise_models[np.argmax(likelihoods)]  # Choose best noise model

# Reconstruct psi_0 using backward evolution
estimated_psi_0 = hamiltonian_evolution(psi_f_noisy, best_Hn, H0, backwards=True)

# Compute fidelity with true |ψ₀⟩
fidelity_reconstruction = np.abs(np.dot(estimated_psi_0.conj().T, true_psi_0))**2

# Print results
print("\nFidelity with different noise models:", fidelity_scores)
print("\nBest noise model chosen:", np.argmax(likelihoods))
print("\nEstimated |ψ₀⟩:\n", estimated_psi_0)
print("\nReconstruction Fidelity:", fidelity_reconstruction)



Fidelity with different noise models: [0.8849403888040731, 0.9597156753936994, 0.9999999999999996]

Best noise model chosen: 2

Estimated |ψ₀⟩:
 [ 1.29906173e-04-3.67901553e-04j -1.61306888e-02+1.13868067e-02j
  6.29851378e-06-4.44618081e-06j -3.90162220e-04-7.80325480e-07j
  6.29851378e-06-4.44618081e-06j -3.90162220e-04-7.80325480e-07j
  1.52345765e-07+3.04691929e-10j -6.28067871e-06-4.47133923e-06j
 -1.61306888e-02+1.13868067e-02j  9.99217523e-01+1.99843771e-03j
 -3.90162220e-04-7.80325480e-07j  1.60850126e-02+1.14512382e-02j
 -3.90162220e-04-7.80325480e-07j  1.60850126e-02+1.14512382e-02j
 -6.28067871e-06-4.47133923e-06j  1.28433532e-04+3.68418233e-04j]

Reconstruction Fidelity: 0.9984396524529728


In [3]:
import numpy as np
from scipy.linalg import expm

# Number of qubits and Hilbert space dimension
n_qubits = 4
dim = 2 ** n_qubits  
i = 9  # Encoding for psi_0

# Initial state |ψ₀⟩
true_psi_0 = np.zeros(dim, dtype=np.complex64)
true_psi_0[i] = 1

# Define Pauli matrices
X = np.array([[0, 1], [1, 0]], dtype=np.complex64)
Z = np.array([[1, 0], [0, -1]], dtype=np.complex64)

# Define original and final Hamiltonians
H0 = sum(np.kron(np.kron(np.eye(2**i), Z), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))
Hf = sum(np.kron(np.kron(np.eye(2**i), X), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))

# Function for time evolution
def time_evolution(H, dt):
    return expm(-1j * H * dt)

def hamiltonian_evolution(psi, H_init, H_final, backwards=False, T=1.0, steps=50):
    dt = T / steps
    psi_t = psi.copy()
    for step in range(steps):
        s = step / steps
        H = (1 - s) * H_init + s * H_final
        U = time_evolution(H, dt if not backwards else -dt)
        psi_t = U @ psi_t
    return psi_t / np.linalg.norm(psi_t)  # Normalize

# Evolve forward to get |ψ_f⟩
psi_f = hamiltonian_evolution(true_psi_0, H0, Hf)

# Generate multiple noisy versions of Hf
noise_levels = [0.1, 0.2, 0.3]  # Different noise intensities
noise_models = [Hf + noise * np.random.randn(*Hf.shape) for noise in noise_levels]

# Superpose noisy evolutions
coefficients = np.array([1/np.sqrt(len(noise_models))] * len(noise_models))  # Equal weighting

# Apply noisy evolution to generate a superposition state
psi_f_noisy_superposed = sum(c * hamiltonian_evolution(true_psi_0, H0, Hn) for c, Hn in zip(coefficients, noise_models))
psi_f_noisy_superposed /= np.linalg.norm(psi_f_noisy_superposed)

# Compute likelihoods based on overlap with final observed state
likelihoods = np.array([
    np.abs(np.vdot(hamiltonian_evolution(true_psi_0, H0, Hn), psi_f_noisy_superposed))**2
    for Hn in noise_models
])

# Normalize likelihoods (Bayesian update)
posterior = likelihoods / np.sum(likelihoods)

# Choose best noise model
best_Hn_index = np.argmax(posterior)
best_Hn = noise_models[best_Hn_index]

# Reconstruct initial state using best noise model
estimated_psi_0 = hamiltonian_evolution(psi_f_noisy_superposed, best_Hn, H0, backwards=True)
fidelity = np.abs(np.dot(estimated_psi_0.conj().T, true_psi_0))**2

# Print results
print("\nPosterior Probabilities for Noise Models:", posterior)
print("\nBest Noise Model Index:", best_Hn_index)
print("\nReconstructed Initial State |ψ₀⟩:\n", estimated_psi_0)
print("\nFidelity with True |ψ₀⟩:", fidelity)



Posterior Probabilities for Noise Models: [0.34445819 0.34035756 0.31518425]

Best Noise Model Index: 0

Reconstructed Initial State |ψ₀⟩:
 [ 0.00419915-0.05987233j -0.05071118+0.00028052j  0.01842616-0.03363182j
  0.01215339+0.10850574j  0.03513213-0.0173104j  -0.05141758+0.00056275j
 -0.01136827-0.05847794j -0.00927167+0.05775696j -0.08548304+0.04650743j
  0.96955925+0.07031071j  0.01879349+0.01707749j -0.02151971-0.0022739j
 -0.0196608 +0.00274383j  0.03317008+0.10130874j  0.01534301+0.00701667j
  0.03187414-0.02559069j]

Fidelity with True |ψ₀⟩: 0.944988734793824


In [4]:
import numpy as np
from scipy.linalg import expm

# Number of qubits and Hilbert space dimension
n_qubits = 4
dim = 2 ** n_qubits  
i = 9  # Encoding for psi_0

# Initial state |ψ₀⟩
true_psi_0 = np.zeros(dim, dtype=np.complex64)
true_psi_0[i] = 1

# Define Pauli matrices
X = np.array([[0, 1], [1, 0]], dtype=np.complex64)
Z = np.array([[1, 0], [0, -1]], dtype=np.complex64)

# Define original and final Hamiltonians
H0 = sum(np.kron(np.kron(np.eye(2**i), Z), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))
Hf = sum(np.kron(np.kron(np.eye(2**i), X), np.eye(2**(n_qubits-i-1))) for i in range(n_qubits))

# Function for time evolution
def time_evolution(H, dt):
    return expm(-1j * H * dt)

def hamiltonian_evolution(psi, H_init, H_final, backwards=False, T=1.0, steps=50):
    dt = T / steps
    psi_t = psi.copy()
    for step in range(steps):
        s = step / steps
        H = (1 - s) * H_init + s * H_final
        U = time_evolution(H, dt if not backwards else -dt)
        psi_t = U @ psi_t
    return psi_t / np.linalg.norm(psi_t)  # Normalize

# Evolve forward to get |ψ_f⟩
psi_f = hamiltonian_evolution(true_psi_0, H0, Hf)

# Generate multiple noisy versions of Hf
noise_levels = [0.1, 0.2, 0.3]  # Different noise intensities
noise_models = [Hf + noise * np.random.randn(*Hf.shape) for noise in noise_levels]

# Superpose noisy evolutions
coefficients = np.array([1/np.sqrt(len(noise_models))] * len(noise_models))  # Equal weighting

# Apply noisy evolution to generate a superposition state
psi_f_noisy_superposed = sum(c * hamiltonian_evolution(true_psi_0, H0, Hn) for c, Hn in zip(coefficients, noise_models))
psi_f_noisy_superposed /= np.linalg.norm(psi_f_noisy_superposed)

# Compute likelihoods based on overlap with final observed state
likelihoods = np.array([
    np.abs(np.vdot(hamiltonian_evolution(true_psi_0, H0, Hn), psi_f_noisy_superposed))**2
    for Hn in noise_models
])

# Normalize likelihoods (Bayesian update)
posterior = likelihoods / np.sum(likelihoods)

# Choose best noise model
best_Hn_index = np.argmax(posterior)
best_Hn = noise_models[best_Hn_index]

# Use best noise model to evolve backward
estimated_psi_0 = hamiltonian_evolution(psi_f_noisy_superposed, best_Hn, H0, backwards=True)

# Define Grover search components
basis_states = np.eye(dim, dtype=np.complex64)

# Define uniform superposition state |s⟩
psi_s = np.ones(dim, dtype=np.complex64) / np.sqrt(dim)

# Construct Diffusion Operator: D = 2|s⟩⟨s| - I
diffusion = 2 * np.outer(psi_s, psi_s.conj()) - np.eye(dim)

# Define Grover Oracle using estimated psi_0
oracle = np.eye(dim) - 2 * np.outer(estimated_psi_0, estimated_psi_0.conj())

# Number of Grover Iterations (~π/4 * sqrt(N))
num_iterations = int(np.floor(np.pi / 4 * np.sqrt(dim)))

# Initialize search state in |s⟩ (equal superposition of all states)
psi_search = psi_s.copy()

# Apply Grover iterations
for _ in range(num_iterations):
    psi_search = oracle @ psi_search  # Apply Oracle
    psi_search = diffusion @ psi_search  # Apply Diffusion

# Measure the final state by finding the largest amplitude
winning_index = np.argmax(np.abs(psi_search)**2)

# Reconstruct psi_0
winning_state = np.zeros(dim, dtype=np.complex64)
winning_state[winning_index] = 1

# Compute Fidelity with true |ψ₀⟩
fidelity = np.abs(np.dot(winning_state.conj().T, true_psi_0))**2

# Print results
print("\nPosterior Probabilities for Noise Models:", posterior)
print("\nBest Noise Model Index:", best_Hn_index)
print("\nEstimated |ψ₀⟩ from Bayesian Inference:\n", estimated_psi_0)
print("\nGrover's Search Final State:\n", winning_state)
print("\nFidelity with True |ψ₀⟩:", fidelity)



Posterior Probabilities for Noise Models: [0.35504718 0.34060761 0.30434521]

Best Noise Model Index: 0

Estimated |ψ₀⟩ from Bayesian Inference:
 [ 0.04178232+0.03724766j -0.07712725+0.01610713j  0.05017603-0.05752911j
  0.04259342-0.08461558j  0.06115053-0.09398548j -0.02443166+0.08630683j
 -0.020053  +0.03415805j  0.00326845+0.05869475j -0.02607603+0.00404189j
  0.9626238 +0.04830861j  0.00522099-0.02895246j -0.02165921-0.02162178j
  0.04837394+0.06599899j -0.02525553+0.0749013j  -0.03113828+0.0663038j
 -0.01138159-0.01727689j]

Grover's Search Final State:
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

Fidelity with True |ψ₀⟩: 1.0
